# How to style Buckaroo tables
Buckaroo had a major refactoring of the styling system and callbacks with the 0.6 release.

This notebook walks through
* Styling columns via the `displayer`
* How to override columns
* Tooltips
* Cell Coloring
* Column hiding
* pinned_rows
* How to add automatic styling methods that are available via the UI to cycle through

In [ ]:
import pandas as pd
import numpy as np
from buckaroo.dataflow import StylingAnalysis
from buckaroo.pluggable_analysis_framework.pluggable_analysis_framework import ColAnalysis
import polars as pl
from buckaroo.polars_buckaroo import PolarsBuckarooWidget

In [ ]:
ROWS = 200
typed_df = pl.DataFrame({'int_col':np.random.randint(1,50, ROWS), 'float_col': np.random.randint(1,30, ROWS)/.7,
                         "str_col": ["foobar"]* ROWS})

In [ ]:
PolarsBuckarooWidget(typed_df)

## `displayer`
Changing the `displayer` is the most common way to customize the styling of a column, in the next example, we override the column_config for `float_col`


In [ ]:
bw2 = PolarsBuckarooWidget(
    typed_df, 
    debug=False,
    column_config_overrides={
        'float_col':
            {'displayer_args': { 'displayer': 'float', 'min_fraction_digits':0, 'max_fraction_digits':3}}})
bw2

Now we are going to force `float_col` to be displayed with a 'float' displayer
notice how the decimal point aligns as opposed to above where 10 is floored without a decimal portion

Currently the types are best viewed in their typescript definition [DFWhole.ts](https://github.com/paddymul/buckaroo/blob/feat/dfviewer-config/js/components/DFViewerParts/DFWhole.ts)

There are Displayers of

`ObjDisplayer`, `BooleanDisplayer`, `StringDisplayer`, `FloatDisplayer`, 
`DatetimeDefaultDisplayer`, `DatetimeLocaleDisplayer`, `IntegerDisplayer`,

`HistogramDisplayer`, and `LinkifyDisplayer`,

There are planned displayers of [HumanAbbreviationDisplayer](https://github.com/paddymul/buckaroo/issues/83), [LineChartDisplayer](https://github.com/paddymul/buckaroo/issues/210), [GoogleMapsLinkDisplayer](https://github.com/paddymul/buckaroo/issues/211) , [InlineMapDisplayer](https://github.com/paddymul/buckaroo/issues/212)


There is experimental work building mirrored PyDantic types, and work to integrate this typechecking into Buckaroo.  There are also plans for a gallery of examples of the different options.

# `tooltip_config`

There are tooltip_configs of simple summary_series available

Tooltips are helpful for adding extra context to cells.  Particularly for noting errors or values changed via auto-cleaning

Notice that `column_config_overrides` is merged with the existing column config from Buckaroo, every column still has a displayer

In [ ]:
bw3 = PolarsBuckarooWidget(
    typed_df, 
    column_config_overrides={
        'str_col':
            {'tooltip_config': { 'tooltip_type':'simple', 'val_column': 'int_col'}}})
bw3

# color_map_config

Color_map_config controls coloring of columns.  
* `color_map` uses the bins from histogram to show a values place in the distribution.  wit the `val_column` parameter, you can color one column based on another.
* `color_when_not_null` hilights a cell when another row is not null.  This is meant for error highlighting,  the other column can be hidden
* `color_from_column` bases the color of a cell based on the RGB value written to another column.  It is the most generic coloring option

In [ ]:
bw3 = PolarsBuckarooWidget(
    typed_df, 
    column_config_overrides={
        'float_col': {'color_map_config': {
          'color_rule': 'color_map',
          'map_name': 'BLUE_TO_YELLOW',
        }}})
bw3

In [ ]:
bw3 = PolarsBuckarooWidget(
    typed_df, 
    column_config_overrides={
        'int_col': {'color_map_config': {
            'color_rule': 'color_map',
            'map_name': 'DIVERGING_RED_WHITE_BLUE',
            'val_column': 'float_col'
        }}})
bw3

# Hiding a column

You can hide a column with `merge_rule:'hidden'`.  This removes that column from the column_config array.

Column hiding can be used to keep data in a dataframe (sent to the table widget) for use as a tooltip, or color, but preventing display which would distract the user


In [ ]:
bw_ = PolarsBuckarooWidget(
    typed_df, 
    column_config_overrides={
        'int_col': {'merge_rule': 'hidden'}})
bw_

# Pinned rows
Pinned rows are visible in each view.  They read data from the summary data that is assembled from the [PluggableAnalysisFramework](https://buckaroo-data.readthedocs.io/en/latest/articles/pluggable.html).


In [ ]:
bw = PolarsBuckarooWidget(
    typed_df, 
    pinned_rows=[
        {'primary_key_val': 'dtype',     'displayer_args': {'displayer': 'obj' } },
        {'primary_key_val': 'histogram', 'displayer_args': {'displayer': 'histogram' }},   
    ])
bw

# Packaging and reusing styling configurations with the `StylingAnalysis` class
Up to this point we have been hardcoding config overrides.  You can package sets of styling configuration with the `StylingAnalysis` class.  These configs can then be toggled in the UI.

If you just want to return a fixed `DFViewerConfig` override the `style_columns` method.  Most likely though, you will want to overide the singular `style_column` method that gets `SingleColumnMetadata` and returns a `ColumnConfig`. 
The `StylingAnalysis` class is used to control the display of a column based on the column metadata.  

return {'col_name':str(col), 'displayer_args': {'displayer': 'obj'}}


This lets you customize based on metadata collected about a column.  This works with the [PluggableAnalysisFramework](https://buckaroo-data.readthedocs.io/en/latest/articles/pluggable.html),  you can specify required fields that are necessary.  Adding requirements like this garuntees that errors are spotted early.

StylingAnalysis works for both Polars and Pandas because it only receives a dictionary with simple python values

In [ ]:
class SimpleStylingAnalysis(StylingAnalysis):
    pinned_rows = [{'primary_key_val': 'dtype', 'displayer_args': {'displayer': 'obj'}}]

    #typing is still be worked out
    #def style_column(col:str, column_metadata: SingleColumnMetadata) -> ColumnConfig:
    @classmethod
    def style_column(kls, col, column_metadata):
        return {'col_name':str(col), 'displayer_args': {'displayer': 'obj'}}

        
    #what is the key for this in the df_display_args_dictionary
    df_display_name = "MyStyles"
def obj_(pkey):
    return {'primary_key_val': pkey, 'displayer_args': { 'displayer': 'obj' } }
def float_(pkey, digits=3):
    return {'primary_key_val': pkey, 
            'displayer_args': { 'displayer': 'float', 'min_fraction_digits':digits, 'max_fraction_digits':digits}}

## Toggling between styles
In the following example, you can cycle between the two configs by clicking on "main".  Eventually I will add a Select (dropdown) box for this.  I personally prefer click to cycle

In [ ]:
#The following analysis only
class SummaryStatsAnalysis(StylingAnalysis):
    requires_summary = ['dtype', 'min', 'mode', 'mean', 'max', 'unique_count', 'distinct_count', 'empty_count']
    pinned_rows = [obj_('dtype'), float_('min'), 
                   #float_('mode'), float_('mean'),  
                   float_('max'), float_('unique_count', 0),
                   float_('distinct_count', 0), float_('empty_count', 0)]
    df_display_name = "summary"
    data_key = "empty"
    summary_stats_key= 'all_stats'

sbw = PolarsBuckarooWidget(typed_df, debug=True)
sbw.add_analysis(SummaryStatsAnalysis)

sbw

## overriding styling from post_processing

`PostProcessing` returns a tuple of Dataframe, extra column metadata.

The default base styling class which all examples here extend, handles the key of `column_config_override` speically.

This lets you build  a specialized dataframe along with specific styling rules.  Auto_cleaning and regular polars/pandas analysis can use the same facility. It is not recommended for regular analysis.



In [ ]:
class ColumnConfigOverride(ColAnalysis):
    @classmethod
    def post_process_df(kls, df):
        return [df, {
            'int_col':{
                'column_config_override': {'color_map_config': {
                    'color_rule': 'color_map',
                    'map_name': 'BLUE_TO_YELLOW',
                }}}}]
    post_processing_method = "override"
bw = PolarsBuckarooWidget(typed_df)
bw.add_analysis(ColumnConfigOverride)
bw            

# Buckaroo internals related to styling
this can help debug what is going on with styling

In [ ]:
#you can view the main display_args with the following statement,  this lets you check what is actually being sent to the frontend
sbw.df_display_args['main']['df_viewer_config']

In [ ]:
# it's annoying to type out all of those pinned rows, lets make some convienence functions
def obj_(pkey):
    return {'primary_key_val': pkey, 'displayer_args': { 'displayer': 'obj' } }

def float_(pkey, digits=3):
    return {'primary_key_val': pkey, 
            'displayer_args': { 'displayer': 'float', 'min_fraction_digits':digits, 'max_fraction_digits':digits}}

class SummaryStatsAnalysis1(SimpleStylingAnalysis):
    pinned_rows = [
        { 'primary_key_val': 'dtype',     'displayer_args': { 'displayer': 'obj' } },
        { 'primary_key_val': 'histogram', 'displayer_args': { 'displayer': 'histogram' }},   
    ]
    df_display_name = "summary1"
    data_key = "empty"
    summary_stats_key= 'all_stats'
class SummaryStatsAnalysis(SimpleStylingAnalysis):
    pinned_rows = [
        obj_('dtype'),
        float_('min'),
        float_('mean'),
        float_('max'),
    ]
    df_display_name = "summary"
    data_key = "empty"
    summary_stats_key= 'all_stats'
base_a_klasses = PolarsBuckarooWidget.analysis_klasses.copy()
base_a_klasses.extend([SummaryStatsAnalysis1, SummaryStatsAnalysis])
class SummaryBuckarooWidget(PolarsBuckarooWidget):
    analysis_klasses = base_a_klasses
sbw = SummaryBuckarooWidget(typed_df)
#also lets do some hacking so that we start with the summary stats view
bstate = sbw.buckaroo_state.copy()
bstate['df_display'] = 'summary1'
sbw.buckaroo_state= bstate
sbw

In [ ]:
class SummaryStatsAnalysis(SimpleStylingAnalysis):
    pinned_rows = [
        obj_('dtype'),
        float_('min'),
        #float_('median'),
        float_('mean'),
        float_('max'),
        float_('unique_count', 0),
        float_('distinct_count', 0),
        float_('empty_count', 0)
    ]
    df_display_name = "summary"
    data_key = "empty"
    summary_stats_key= 'all_stats'
base_a_klasses = PolarsBuckarooWidget.analysis_klasses.copy()
base_a_klasses.append(SummaryStatsAnalysis)
class SummaryBuckarooWidget(PolarsBuckarooWidget):
    analysis_klasses = base_a_klasses
sbw = SummaryBuckarooWidget(typed_df)
#also lets do some hacking so that we start with the summary stats view
bstate = sbw.buckaroo_state.copy()
bstate['df_display'] = 'summary'
sbw.buckaroo_state= bstate
sbw

In [ ]:
class AdaptiveStyling(StylingAnalysis):
    requires_summary = ["histogram", "is_numeric", "dtype", "is_integer"]
    pinned_rows = [
        obj_('dtype'),
        {'primary_key_val': 'histogram', 'displayer_args': { 'displayer': 'histogram' }}]

    @classmethod
    def style_columns(kls, col, sd):
        digits = 3
        if sd['is_integer']:
            disp = {'displayer': 'float', 'min_fraction_digits':0, 'max_fraction_digits':0}
        elif sd['is_numeric']:
            disp = {'displayer': 'float', 'min_fraction_digits':digits, 'max_fraction_digits':digits}
        else:
            disp = {'displayer': 'obj'}
        return {'col_name':col, 'displayer_args': disp }
base_a_klasses = PolarsBuckarooWidget.analysis_klasses.copy()
base_a_klasses.extend([AdaptiveStyling, ValueCountPostProcessing])
class ABuckarooWidget(PolarsBuckarooWidget):
    analysis_klasses = base_a_klasses
acb = ABuckarooWidget(typed_df)
acb